In [0]:
%fs ls /mnt/Anac/Silver/

In [0]:
df = spark.read.parquet("dbfs:/mnt/Anac/Silver/anac_silver.parquet/", header=False, inferSchema=True)
display(df)

In [0]:
#1°:Selecionar somentes colunas que o cliente/setor pediu
colunas = ['Aerodromo_de_Destino', 'Aerodromo_de_Origem','Classificacao_da_Ocorrência', 'Danos_a_Aeronave', 'Data_da_Ocorrencia','Municipio','UF','Regiao','Tipo_de_Aerodromo', 'Tipo_de_Ocorrencia','Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']

df = df.select(colunas)

display(df)

In [0]:
#CRIAR NOVA COLUNA QUE FAÇA A SOMA DAS LESÕES
colunas_lesoes = ['Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']


df = df.withColumn("TOTAL_LESOES", sum(df[colunas_lesoes]))

display(df)

In [0]:
#RENOMEAR COLUNAS PARA FICAR MAIS INTUITIVAS

novos_nomes = ['Destino', 'Origem', 'Classificacao', 'Danos', 'Data','Municipio','UF','Regiao','Tipo_de_Aerodromo', 'Tipo_de_Ocorrencia','Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros','Lesoes_Graves_Terceiros','Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes', 'Total de lesões']

for i in range(0, len(df.columns)):
    df = df.withColumnRenamed(df.columns[i], novos_nomes[i])

df.columns

In [0]:
from pyspark.sql.functions import col
#EXCLUIR DADOS QUE TENHAM A CLASSIFICAÇÃO [Indeterminado, Sem Registro, Exterior]
valores_para_excluir = ["Indeterminado", "Sem Registro", "Exterior"]

for coluna in df.columns:
    df = df.filter(~col(coluna).isin(valores_para_excluir))

display(df)

In [0]:
from pyspark.sql.functions import current_timestamp, expr

#INSERIR COLUNA PARA USUARIO VER QUANDO FOI ATUALIZADO
df = df.withColumn("Data_de_atualização", expr("from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')"))

display(df)

In [0]:
#SALVAR DATA FRAME NA CAMADA GOLD PARTICIONANDO POR ESTADO

df.write\
    .partitionBy("UF")\
    .format("parquet")\
    .mode("overwrite")\
    .parquet('dbfs:/mnt/Anac/Gold/anac_gold.parquet/')